In [171]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from docx import Document
import gradio as gr 
import os
import requests
import json

load_dotenv(override=True)


True

In [172]:
Push_over_user = os.getenv("PUSHOVER_USER_KEY")
Push_over_API_Token = os.getenv("PUSHOVER_API_TOKEN")
PushOver_URL = "https://api.pushover.net/1/messages.json"

In [173]:
def notifyUnansweredQuestion(Question):
     payload = {"user": Push_over_user, "token": Push_over_API_Token, "message": Question}
     response = requests.post(url=PushOver_URL,data=payload)
     #print(Question)
     return {"recorded":"Ok"}
    

In [174]:
def notifyUserInfo(email):
    data = {"token": Push_over_API_Token,"user": Push_over_user,"message": f"user {email} is intreasted to discuss with you..!"}
    requests.post(url=PushOver_URL,data=data)
    #print(email)
    return {"recorded":"Ok"}

In [175]:
notifyUnansweredQuestionTool={
    "name":"NotifyUnansweredQuestion",
    "Description":"This function is used to notify Question asked. when you dont know the answer.",
    "type":"Object",
    "Parameter":{
        "name":"Question",
        "type":"String"
    },
    "required":["email"]
}

notifyUserInfoTool={
    "name":"notifyUserInfo",
    "Description": "This fuction will notify the user's email. who is interested to reach me.",
    "type":"Object",
    "parameter":{
        "name":"email",
        "type":"string"
    },
    "required":["email"]
}
Tool=[{"type":"function", "function":notifyUnansweredQuestionTool},
      {"type":"function", "function":notifyUserInfoTool}]

In [176]:
def handelfunction(tool_calls):

     for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!
        results = []
        if tool_name == "notifyUnansweredQuestionTool":
            result = notifyUnansweredQuestion(**arguments)
        elif tool_name == "notifyUserInfoTool":
            result = notifyUserInfo(**arguments)

    # results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_calls.id})

In [177]:
pdfreader = PdfReader("./me/Profile.pdf")
LinkedInProfile = ""
for page in pdfreader.pages: 
        LinkedInProfile += page.extract_text()

summary = ""
doc = Document("./Me/Summary.docx")
for para in doc.paragraphs:
    summary += para.text

In [178]:
name = "Srinivasan Amaranathan"

SystemPrompt = f"You are acting as {name}.What ever Question raised relared to professional life you should be able to answer\
                as {name} . i am sharing you both resume and Linked profile.\
                Resume is {summary} and \
                Linked profile is {LinkedInProfile}.\
                Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
                Notify the user details such as email id who are initiating the Chat.\
                if you dont know the answer notify me the question."

SystemPrompt += f"\n\n## Resume:\n{summary}\n\n## LinkedIn Profile:\n{LinkedInProfile}\n\n"
SystemPrompt += f"With this context, please chat with the user, always staying in character as {name}."

In [179]:
def chat(message, history):
    client = OpenAI(base_url="http://localhost:11434/v1",api_key="Ollama")
    message = [{"role":"system","content":SystemPrompt}]+history+[{"role":"user","content":message}]
    done = "true"

    while done == "true":
      response = client.chat.completions.create(model="llama3.2:3b",
                                    messages=message,
                                  tools=Tool)
      print(response.choices[0].finish_reason)
      if response.choices[0].finish_reason == "tool_calls":
         results = handelfunction(response.choices[0].message.tool_calls)
         message.append(message)
         message.extend(results)
      else:
         done = "False"

    return response.choices[0].message.content
        
    

In [180]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


tool_calls
Tool called: notifyUserInfo


Traceback (most recent call last):
  File "c:\srini\UdemyAgent\MyPratices\.venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\srini\UdemyAgent\MyPratices\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\srini\UdemyAgent\MyPratices\.venv\Lib\site-packages\gradio\blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\srini\UdemyAgent\MyPratices\.venv\Lib\site-packages\gradio\blocks.py", line 1738, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\srini\UdemyAgent\MyPratices\.venv\Lib\site-packages\gradio\utils.py", line 907, in async_wrapper
    response = await f(*args, **kwargs)
    